In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
import pyspark.sql.functions as F
import json

conf = SparkConf().setAppName('MyFirstStandaloneApp')
sc = SparkContext(conf=conf)

filepath = "./flights_reduces.text "

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
filtered = flightsRDD.filter(lambda line: line != header)
parts = filtered.map(lambda line: line.split(';'))
# Robilem to na roboczym pliku, trzeba sprawdzic czy finalnie pokrywa sie ta kolejnosc kolumn
flights = parts.map(lambda p:
                   Row(Scrap_date = p[0],
                       Scrap_time = p[1],
                       Country_from = p[2],
                       Country_to = p[3],
                       Flight_id = int(p[4]),
                       Flight_date = p[5],
                       Airline = p[6],
                       Change = int(p[7]),
                       Price = float(p[8]),
                       Depart_hour = p[9],
                       Depart_from = p[10],
                       Arrival_hour = p[11],
                       Arrive_to = p[12]
                      ))

sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(flights)
# To days cos nie dziala, tu musze poprawic, wczesniej dzialo, ale nie na wersji 1.6 i wrzucilem jakis kod ze stacka,
# ale wyrzuca null
dt = dt.withColumn("Days", F.datediff(F.to_date(F.unix_timestamp( dt.Flight_date, "dd/MM/yy").cast("timestamp")), 
                                    F.to_date(F.unix_timestamp( dt.Scrap_date, "'yyyy-MM-dd").cast("timestamp"))))
# Tutaj sobie dodaje kolumne roboczo ThereBack, w naszym pliku to bedzie, wiec mozna usunac
dt = dt.withColumn("ThereBack", F.lit('There'))
dt.registerTempTable("flights")
dt.show()

+-----------+------------+--------------------+------+------------+----------+--------------------+-----------+-----------+---------+-----+----------+----------+----+---------+
|    Airline|Arrival_hour|           Arrive_to|Change|Country_from|Country_to|         Depart_from|Depart_hour|Flight_date|Flight_id|Price|Scrap_date|Scrap_time|Days|ThereBack|
+-----------+------------+--------------------+------+------------+----------+--------------------+-----------+-----------+---------+-----+----------+----------+----+---------+
|   Wizz Air|       04:10|         Bourgas BOJ|     1|     Austria|  Bulgaria|London LTNLondon ...|      22:40|   14/06/19|       35|70.15|2019-05-07|     05:00|null|    There|
|   Wizz Air|       07:15|London LTNLondon ...|     1|     Austria|  Bulgaria|         Bourgas BOJ|      05:25|   21/06/19|       35|20.45|2019-05-07|     05:00|null|    There|
|    easyJet|       16:10|          Vienna VIE|     1|     Austria|  Bulgaria|London LTNLondon ...|      12:45|   2

In [2]:
price_sum.write.format('com.databricks.spark.csv').save('Price_sum.csv')

Tutaj poniżej jest taki przykładowy kod, jak wykonac filtrowanie np. zsumowac wszystkie ceny biletow w pliku

In [4]:
# To mozna w jakims jsonie umiescic jako plik konfiguracyjny, ktory tam sie zmienia
direction = 'There'
country_from = 'Austria'
country_to = 'Bulgaria'
scrap_time = '05:00'

query = "SELECT * FROM flights WHERE Country_from = '{0}' AND Country_to = '{1}' \
AND Scrap_time = '{2}' AND ThereBack = '{3}'".format(country_from, country_to, scrap_time, direction)

test = sqlContext.sql(query)
price_sum = test.agg(F.sum(dt.Price).alias('Price_sum'))
#price_sum.rdd.map(lambda x: x).saveAsTextFile('Price_sum.csv')
# Uwaga, tutaj jest problem, bo dziala mi to na lapku, ale nie dziala na tej maszynie wirtualnej... to zapisywanie do csv
# No i oddziela przecinkami, a nie srednikami, tego trzeba poszukac
# Jak raz wywolacie zapisywanie do jakiegos pliku to potem jak chcecie to odpalic jeszcze raz to pamietajcie, zeby
# Usunac ten stary plik/folder co sie wygeneruje, bo wam wysypie bledy
price_sum.write.format('com.databricks.spark.csv').save('Price_sum_1.csv')

In [5]:
query_proc = "SELECT * FROM flights WHERE Country_from = '{0}' AND Country_to = '{1}'".format(country_from, country_to)
processed = sqlContext.sql(query_proc)

Patrzę w ten sposób: chcę polecieć w podróż A-B, nieważne na jak długo, nie ważne kiedy jest lot, liczy się tylko cena przelotu z podziałem na czas kiedy taka występuje. Tutaj poniżej robię taki myk, że grupuję sobie po flight_id i patrzę jaka jest skumulowana cena każdej podróży. Później robię grupowanie po Scrap_date i Scrap_time, tak że dostaję tabelę z najniższą ceną podróży każdego dnia o porze skrapowania. Wydaje mi się, że jest to dobra tabela, żeby na jej podstawie zrobić jakąś regresję oraz wyplotować dane historyczne po dniach.
W analogii do dplyr:
groupBy to samowyjaśnialne, a agg to takie summarise

In [7]:
df = processed.groupBy(["Flight_id"]).agg(F.first(processed.Scrap_date).alias('Scrap_date'), 
                                            F.first(processed.Scrap_time).alias('Scrap_time'), 
                                             F.sum(processed.Price).alias("Price"))
df = df.groupBy(["Scrap_date", "Scrap_time"]).agg(F.min(df.Price).alias('Price'))
df.show()
df.write.format('com.databricks.spark.csv').save('Regression2.csv')

+----------+----------+-----------------+
|Scrap_date|Scrap_time|            Price|
+----------+----------+-----------------+
|2019-05-07|     05:00|4367.999999999997|
+----------+----------+-----------------+



Poniżej robię taką ogólną tabelę, która jest odtworzeniem tej generalnej tabeli ze strony azair.eu

In [10]:
df2 = processed.groupBy(["Flight_id", "Direction"]).agg(
    F.sum(processed.Price).alias("Price"),
    F.first(processed.Scrap_date).alias("Scrap_date"),
    F.first(processed.Flight_date).alias("Flight_date"),
    F.first(processed.Country_from).alias("Country_from"),
    F.first(processed.Country_to).alias("Country_to"),
    F.first(processed.Depart_hour).alias("Depart_hour"),
    F.first(processed.Depart_from).alias("Depart_from"),
    F.last(processed.Arrival_hour).alias("Arrival_hour"),
    F.last(processed.Arrive_to).alias("Arrive_to"),
)

In [11]:
df2.show()

+---------+---------+-----------------+----------+-----------+------------+----------+-----------+--------------------+------------+--------------+
|Flight_id|Direction|            Price|Scrap_date|Flight_date|Country_from|Country_to|Depart_hour|         Depart_from|Arrival_hour|     Arrive_to|
+---------+---------+-----------------+----------+-----------+------------+----------+-----------+--------------------+------------+--------------+
|       35|    There|4367.999999999997|2019-05-07|   14/06/19|     Austria|  Bulgaria|      22:40|London LTNLondon ...|       17:25|Klagenfurt KLU|
+---------+---------+-----------------+----------+-----------+------------+----------+-----------+--------------------+------------+--------------+



Podobnie jak przy df1, ale grupuję sobie po Destination jeszcze, tak że mi sie rozbija na loty "There" i "Back"

In [12]:
df3 = processed.groupBy("Scrap_date", "Scrap_time", "Direction").agg(
    F.min(processed.Price).alias("Price"),
)
df3.show()

+----------+----------+---------+-----+
|Scrap_date|Scrap_time|Direction|Price|
+----------+----------+---------+-----+
|2019-05-07|     05:00|    There|11.81|
+----------+----------+---------+-----+

